In [77]:
import numpy as np
from numpy import sin, cos, pi
import matplotlib.pyplot as plt

In [142]:
# Calculation constants
n_grid = 100
n_step = 1
dphi = 2*pi / n_step
phis = np.linspace(0, 2*pi - dphi, n_step)
scale = 1
d = 0.00005
delta = 2 * d

# Initial conditions
a = 1 * scale   # radius of the ring
I_0 = 1         # current in the ring
w = 100         # angular frequency of the current
c = 10000       # speed of light in CGS units


In [79]:
# Fields
A = np.zeros((n_grid, n_grid, n_grid, 3), dtype=float) # Magnetic vector potential
E = np.copy(A)                                                  # Electric vector potential
B = np.copy(A)                                                  # Magnetic field

In [80]:
# Utility functions
def distance(x, y, z, phi):
    return np.sqrt( (x - a * cos(phi))**2 + (y - a * sin(phi))**2 + z**2 )

def ikj_to_cartesian(i, j, k):
    x = (i - 50) * scale
    y = (j - 50) * scale
    z = (k - 50) * scale
    return x, y, z

In [41]:
# Update A field
def update_A(t):
    for i in range(grid_size):
        for j in range(grid_size):
            for k in range(grid_size):
                update_A_cell(i, j, k, t)
    return

# Update A a cell in the A field
def update_A_cell(i, k, j, t):
    x, y, z = ikj_to_cartesian(i, j, k)
    A[i, j, k] = A_integral(x, y, z, t)
    return 

# Calculate the integral of A
def A_integral(x, y, z, t):
    Cell = np.zeros(3)
    for phi in phis:
        Cell += A_integral_step(x, y, z, t, phi)
    return Cell * 10**-8

# Calculate the integral of A for a given phi
def A_integral_step(x, y, z, t, phi):
    e = np.array([-sin(phi), cos(phi), 0.])
    tr = t - distance(x, y, z, phi) / c
    return tr * sin(w*tr) * e * a / distance(x, y, z, phi) * dphi

In [135]:
def calc_A(x, y, z, t):
    phi_steps = 100
    dphi = 2 * pi / phi_steps
    A = np.zeros(3)

    # phis = np.linspace(0, 2*pi, phi_steps)
    # for phi in phis:

    for i in range(phi_steps):
        phi = i * dphi
        e = np.array([-sin(phi), cos(phi), 0.])
        tr = t - distance(x, y, z, phi) / c
        A += tr * sin(w*tr) * e / distance(x, y, z, phi) * dphi

    return A * 10**-8

In [158]:
# Their version (for reference)
def Afield(x, y, z, t):
    nphi = n_step
    dp = 2*pi/nphi
    A = np.array([0., 0., 0.])
    r = np.array([x, y, z])
    for ip in range(nphi):
        p = ip * 2 * pi / nphi
        rp = a * np.array([cos(p), sin(p), 0.])
        ep = np.array([-sin(p), cos(p), 0.])
        dist = np.linalg.norm(r - rp)
        tr = t - dist/c
        A += (tr>0) * sin(w*tr) * ep * dp / dist
    return A

def E_field(x, y, z, t):
    delta = 0.0001
    return -(Afield(x, y, z, t + delta) - Afield(x, y, z, t)) / delta

def B_field(x, y, z, t):
    delta = 0.0001
    dxA = (Afield(x + delta, y, z, t) - Afield(x, y, z, t)) / delta
    dyA = (Afield(x, y + delta, z, t) - Afield(x, y, z, t)) / delta
    dzA = (Afield(x, y, z + delta, t) - Afield(x, y, z, t)) / delta
    return np.array([dyA[2] - dzA[1], dzA[0] - dxA[2], dxA[1] - dyA[0]])

In [156]:
n_step = 100
dp = 2*pi/n_step
phis = np.linspace(0, 2*pi-dp, n_step)

def A(x, y, z, t):
    A = np.zeros(3)

    for phi in phis:
        e = np.array([-sin(phi), cos(phi), 0.])
        tr = t - distance(x, y, z, phi) / c
        A += tr * sin(w*tr) * e / distance(x, y, z, phi) * dphi

    return A * 10**-8

# Convert A to E
def E_compute(x, y, z, t):
    E = - diff_A_t(x, y, z, t)
    return E

# Convert A to B
def B_compute(x, y, z, t):
    B = np.zeros(3)
    B[0] = diff_A_y(x, y, z, t)[2] - diff_A_z(x, y, z, t)[1]
    B[1] = diff_A_z(x, y, z, t)[0] - diff_A_x(x, y, z, t)[2]
    B[2] = diff_A_x(x, y, z, t)[1] - diff_A_y(x, y, z, t)[0]
    return B
    
n = 1

def diff_A_x(x, y, z, t):
    return (A(x + d, y, z, t) - A(x - d, y, z, t)) / (n * d)

def diff_A_y(x, y, z, t):
    return (A(x, y + d, z, t) - A(x, y - d, z, t)) / (n * d)

def diff_A_z(x, y, z, t):
    return (A(x, y, z + d, t) - A(x, y, z - d, t)) / (n * d)

def diff_A_t(x, y, z, t):
    return (A(x, y, z, t + d) - A(x, y, z, t - d)) / (n * d)

In [159]:
x, y, z, t = 5, 0, 0, 0.2

print(f'My code:  E: {E_compute(x, y, z, t)}, B: {B_compute(x, y, z, t)}')
print(f'Solution: E: {E_field(x, y, z, t)}, B: {B_field(x, y, z, t)}')

My code:  E: [ 1.18655086e-19 -2.31935094e-06 -0.00000000e+00], B: [ 0.00000000e+00  0.00000000e+00 -9.59833513e-09]
Solution: E: [-4.73796349e-13 -5.15546352e+00 -0.00000000e+00], B: [ 7.43441281e-07  1.51788304e-14 -2.39928183e-02]


In [213]:
n_step = 100
dp = 2*pi/n_step
phis = np.linspace(0, 2*pi-dp, n_step)
a = 1
w = 100
c = 10000
I = 1

def calc_A(x, y, z, t):
    A = np.zeros(3)
    for i in range(n_step):
        phi = i * dp
        e = np.array([-sin(phi), cos(phi), 0.])
        tr = t - distance(x, y, z, phi) / c
        A += tr * sin(w*tr) * e / distance(x, y, z, phi)
    return A * I * a * dp / c**2

def Afield(x, y, z, t):
    dp = 2*pi/n_step
    A = np.array([0., 0., 0.])
    r = np.array([x, y, z])
    for ip in range(n_step):
        p = ip * dp
        rp = a * np.array([cos(p), sin(p), 0.])
        ep = np.array([-sin(p), cos(p), 0.])
        dist = np.linalg.norm(r - rp)
        tr = t - dist/c
        A += (tr>0) * sin(w*tr) * ep / dist
    return A / c**2 * dp

def A(x, y, z, t):
    A = np.zeros(3)
    for phi in phis:
        e = np.array([-sin(phi), cos(phi), 0.])
        tr = t - distance(x, y, z, phi) / c
        A += tr * sin(w*tr) * e / distance(x, y, z, phi) 
    return A * I * a / c**2 * dp

In [214]:
x, y, z, t = 10, 0, 0, 0.1

print(f'Calc_A:  A: {calc_A(x, y, z, t)}')
print(f'Afield:  A: {Afield(x, y, z, t)}')
print(f'A:       A: {A     (x, y, z, t)}')

Calc_A:  A: [-1.5940649e-26 -1.7203876e-11  0.0000000e+00]
Afield:  A: [ 2.45785733e-25 -1.72317772e-10  0.00000000e+00]
A:       A: [-1.5940649e-26 -1.7203876e-11  0.0000000e+00]
